# Launching an simulation of a multi-server queue with priorities

The simulation of an M/M/n system supports various types of input and service distributions, any number of channels. It is possible to specify the maximum length of the queue. By default, it is an infinite queue. In Kendall notation: GI/G/n/r and GI/G/n.

To assign priorities, you need to pass the priority type *prty_type* to the IM

| prty_type |                                     Priority Type                                     |
|:---------:|:----------------------------------------------------------------------:|
|    No     |                                 Without priorities, FIFO                                  |
|    PR     |                 Preemptive resume, with resumption of interrupted request                 |
|    RS     | Preemptive repeat with resampling, service again with a new random duration |
|    RW     |   Preemptive repeat without resampling, service again with the same previous duration    |
|    NP     |                        Non-preemptive, relative priority                         |

To launch the IM requires 4 steps:
- creating an instance of the simulation model
- specifying the input stream
- specifying the service distribution
- running the model

#### Define the number of channels, number of classes, and arrival intensities for each class

In [ ]:
from most_queue.rand_distribution import GammaDistribution

n = 5  # number of channels
k = 3  # number of classes
num_of_jobs = 100000 # number of jobs to simulate
l = [0.2, 0.3, 0.4]
lsum = sum(l)

#### Setting up the initial moments of service distribution.

To verify with numerical methods, it is necessary to specify one more initial moment than the required number of initial moments for the distribution of the time spent by requests in the system. That is, to obtain v1, v2, v3, we need to specify 4 moments for each class of requests.
By using the given moments, we can obtain the parameters of the approximating distribution for setting up the IM. In this case, the service time distribution is specified as a Gamma distribution. To find its two parameters, we require two initial moments.

In [ ]:
# First and second initial moments of service time for each of the 3 classes
b1 = [0.45 * n, 0.9 * n, 1.35 * n]
b2 = [0] * k
coev = 0.577

for i in range(k):
    b2[i] = (b1[i] ** 2) * (1 + coev ** 2)

# Average service time and coefficient of utilization
b_sr = sum(b1) / k
ro = lsum * b_sr / n

print(f'ro = {ro:0.3f}')

# Gamma distribution parameters
params = []
for i in range(k):
    params.append(GammaDistribution.get_params([b1[i], b2[i]]))

print('\nGamma distribution parameters:\n', params)

# Calc 4 moments by the obtained parameters
b = []
for j in range(k):
    b.append(GammaDistribution.calc_theory_moments(params[j], 4))

print('\nInitial moments of service time:\n', b)

ro = 0.810

Gamma distribution parameters:
 [(1.3349526308745838, 3.0036434194678137), (0.6674763154372919, 3.0036434194678137), (0.44498421029152807, 3.0036434194678145)]

Initial moments of service time:
 [[2.25, 6.747953062500001, 25.292546083777786, 113.7474276956009], [4.5, 26.991812250000002, 202.3403686702223, 1819.9588431296145], [6.75, 60.73157756250001, 682.8987442620003, 9213.541643343671]]


#### Import the PriorityQueueSimulator class and create an instance of it.
When creating, specify the number of channels *n* and the number of classes *k*. Also specify the type of priority - *PR* (absolute priority).

In [10]:
from most_queue.sim.priority_queue_sim import PriorityQueueSimulator

qs = PriorityQueueSimulator(n, k, "PR")

#### Set sources and servers parameters.

In [11]:
sources = []
servers_params = []
for j in range(k):
    sources.append({'type': 'M', 'params': l[j]})
    servers_params.append({'type': 'Gamma', 'params': params[j]})

qs.set_sources(sources)
qs.set_servers(servers_params)

#### For or running the IM, we need to call the *run* method and pass the number of jobs

In [12]:
num_of_jobs = 300000
qs.run(num_of_jobs)
v_sim = qs.v
print(v_sim)

Start simulation


Job served:    | 0/100 [00:00<?, ?it/s]3000/300000:   1%|          | 1/100 [00:00<00:01, 51.09it/s]6000/300000:   2%|▏         | 2/100 [00:00<00:01, 50.43it/s]9000/300000:   3%|▎         | 3/100 [00:00<00:01, 50.55it/s]12000/300000:   4%|▍         | 4/100 [00:00<00:01, 50.42it/s]15000/300000:   5%|▌         | 5/100 [00:00<00:01, 50.31it/s]15000/300000:   6%|▌         | 6/100 [00:00<00:01, 49.96it/s]18000/300000:   6%|▌         | 6/100 [00:00<00:01, 49.96it/s]21000/300000:   7%|▋         | 7/100 [00:00<00:01, 49.96it/s]24000/300000:   8%|▊         | 8/100 [00:00<00:01, 49.96it/s]27000/300000:   9%|▉         | 9/100 [00:00<00:01, 49.96it/s]30000/300000:  10%|█         | 10/100 [00:00<00:01, 49.96it/s]33000/300000:  11%|█         | 11/100 [00:00<00:01, 49.96it/s]33000/300000:  12%|█▏        | 12/100 [00:00<00:01, 50.57it/s]36000/300000:  12%|█▏        | 12/100 [00:00<00:01, 50.57it/s]39000/300000:  13%|█▎        | 13/100 [00:00<00:01, 50.57it/s]42000/300000:  14%|█▍        | 14/100 [00:00

Simulation is finished

[[2.2522101694749534, 6.744308028876687, 25.209332022622043], [4.920101879864704, 33.00271718765631, 280.9927953658352], [18.020166975672666, 629.0703418814782, 38733.556903533805]]


#### Numermerical calculation using the method of invariant relations

In [ ]:
from most_queue.theory.queueing_systems.priority.mgn_invar_approx import MGnInvarApproximation
mgn_invar = MGnInvarApproximation(l, b, n)
v_teor = mgn_invar.get_v('PR')

#### Вывод результатов

In [ ]:
from most_queue.general.tables import times_print_with_classes
print("\nComparison of simulation and numeric results (method by invariants of relation) \n"
          "for the time spent in a multi-channel M/M/S queue with priorities")
print(f"Number of channels: {n}\nNumber of classes: {k}\nLoad coefficient: {ro:<1.2f}\n")
print(f"Variation coefficient of service time {coev}\n")
print("PR (preemtive) priority")

times_print_with_classes(v_sim, v_teor, is_w=False)


Comparison of simulation and numeric results (method by invariants of relation) 
for the time spent in a multi-channel M/M/S queue with priorities
Number of channels: 5
Number of classes: 3
Load coefficient: 0.81

Variation coefficient of service time 0.577

PR (preemtive) priority
       Initial moments of soujorn time in the system        
------------------------------------------------------------
           |                       #                       |
    Cls    | --------------------------------------------- |
           |       1       |       2       |       3       |
------------------------------------------------------------
     | Sim |     2.25      |     6.74      |     25.2      |
  1  |------------------------------------------------------
     | Num |     2.27      |     6.85      |     25.8      |
------------------------------------------------------------
     | Sim |     4.92      |      33       |      281      |
  2  |---------------------------------------